# **<span style="color:Purple;">Optical Character Recognition: Google Cloud Vision API</span>**
Tesseract OCR and Google Vision API are two options.


**<span style="color:green;">Tesseract OCR:</span>**
Tesseract is a free, open-source OCR tool. It can be customized and can train it to recognize specific handwriting styles, which can improve accuracy. It requires a lot of preprocessing and setup to achieve high accuracy with handwritten text, making it complex and time-consuming.

**<span style="color:green;">Google Vision API:</span>**
Google Vision API is a cloud-based service known for its high accuracy, especially with handwritten text. It is easy to use and requires minimal setup. Also it offers other image analysis features and can handle large volumes of requests. It also costs money and needs an internet connection, which could be an issue if the service is down or the connection is unstable.

🏅 **<span style="color:green;">Choosing Google Vision API:</span>**
For extracting handwritten details like aircraft journey summaries, Google Vision API is the better choice. Its high accuracy, ease of use, and advanced features justify the cost, making it ideal for projects that need precision and efficiency.

## **<span style="color:purple;">Technical Stack</span>**

**<span style="color:green;">Google Cloud Vision API:</span>** For high-accuracy OCR text extraction

**<span style="color:green;">Python:</span>** For backend service development due to its extensive libraries and frameworks

**<span style="color:green;">PostgreSQL:</span>** For robust, scalable, and reliable data storage

**<span style="color:green;">SQLAlchemy:</span>** SQLAlchemy simplifies database interactions by abstracting raw SQL queries into Python objects and methods, making database operations more intuitive and maintainable. Can switch between different database systems (like PostgreSQL, MySQL, SQLite) with minimal changes to your codebaseIts Object-Relational Mapping (ORM) capabilities map database tables to Python classes, allowing you to perform CRUD(Create, Read,Update, Delete) operations in a more Pythonic way. This approach enhances readability, ensures consistency in managing schema changes, and reduces the need for repetitive SQL code, enabling you to focus on higher-level application logic.

In [2]:
import os
import io
import psycopg2
from PIL import Image, ImageOps, ImageFilter
from google.cloud import vision

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

In [ ]:
# Obtain the Google Cloud Vision API from Google Cloud Platform 
""" Login into GCP Account > APIs & Services > Library > Cloud Vision API > Manage > + Create Credentials > Service Account > """
""" > Fill info into Service Account Name & Service Account Description > Select a Role: As "Owner" > Done """
""" > Create a key > JSON > Download the key file > Save it to the same directory """

# Set up Google Cloud Vision API
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'vision_key_api.json'

### **<span style="color:purple;">Data Acquisition</span>**
The first step involves capturing the data from the "aircraft journey summary" form. Engineers will take a photo of the form using the user-facing app. This photo will be sent to the backend service as an image file for processing.

In [ ]:
# Directory where the images will be saved
UPLOAD_FOLDER = 'uploads/'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

### **<span style="color:purple;">Preprocessing the Image</span>**
a. Converting to grayscale reduce images to black and white simplifying the image, reducing noise and enhancing text contrast. 

b. Applying Gaussian blur smooths the image, reducing small details and noise in order to improve the OCR accuracy.

In [ ]:
# Load and preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)

    # Convert the Image Color to Gray - Black & White
    gray_image = ImageOps.grayscale(image)

    # Applying Gaussian Blur on the Gray Image
    blurred_image = gray_image.filter(ImageFilter.GaussianBlur(radius=2))

    with io.BytesIO() as output:
        blurred_image.save(output, format="PNG")
        return output.getvalue()

### **<span style="color:purple;">Performing OCR using Google Vision API</span>**
Once the image is preprocess, utilize the Google Cloud Vision API to perform OCR. The Vision API extracts the text content from the image, converting the handwritten into machine-readable text. This API is chosen for its robust capabilities in text extraction and accuracy

In [ ]:
# Reference from: https://www.youtube.com/watch?v=hkKKfEqZvn4
def perform_ocr(image_data):
    client = vision.ImageAnnotatorClient()
    with io.open(image_data, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)

    texts = response.text_annotations

    if texts:
        return texts[0].description
    else:
        return None

### **<span style="color:purple;">Text Parsing: Parse the OCR Results</span>**
After obtaining the raw text from the OCR process, the next step is to parse this text to identify and extract relevant fields as the the output of the OCR Results text can be messy and unstructured. The parse_ocr_result function is meant to clean this text and extract specific details such as the aircraft model, registration number, and departure airport. Since I don’t have access to the actual API output, this code is left in a general form because the exact structure of the text is unknown. Regular expressions processing techniques will be employed to structure the extracted text into a well-define format

In [ ]:
# Parse the OCR result
def parse_ocr_result(ocr_result):
    data = {}
    lines = ocr_result.split('\n')
    for line in lines:
    #     if 'Aircraft Model:' in line:
    #         data['aircraft_model'] = XXX
    #     elif 'Registration Number:' in line:
    #         data['registration_number'] = XXX
    #     elif 'Departure Airport:' in line:
    #         data['departure_airport'] = XXX
    #     elif 'Crew:' in line:
    #         data['crew'] = XXX
    #     elif 'Fuel:' in line:
    #         data['fuel'] = XXX
    #     elif 'Load:' in line:
    #         data['load'] = XXX
    # return data

### **<span style="color:purple;">Data Storage: PostgreSQL</span>**
PostgreSQL and SQLite are two options

🏅 **<span style="color:green;">Choosing PostgreSQL:</span>**  SQLite is typically much faster than PostgreSQL, as it is much more lightweight and more straightforward in design. However, PostgreSQL is more reliable and can handle larger volumes of data, making it a better choice for this applications 

The structured data will then be stored in a PostgreSQL database. PostgreSQL is selected for its reliability, scalability, and support for complex queries.

a. Ensures that the information/data is stored accurately and securely, maintaining consistency and reliability

b. Enables efficient querying and reporting on aircraft details

c. Can easily retrieve all flights of a specific aircraft model, analyze fuel consumption trends, or generate summaries based on departure airports

In [ ]:
# Reference from: https://www.youtube.com/watch?v=M2NzvnfS-hI 

# Connecting to PostgreSQL Database - creatimg table and inserting values in it
def insert_into_db(data):
    conn = psycopg2.connect(
        hostname="localhost",
        database="demo",
        username="postgres",
        pwd="admin",
        port_id= 5432
    )
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS aircraft_journey_summary")

    create_script = """
    CREATE TABLE IF NOT EXISTS aircraft_journey_summary (
        id int PRIMARY KEY,
        aircraft_model varchar(50),
        registration_number varchar(50),
        departure_airport varchar(50),
        crew int,
        fuel int,
        load int) """
    
    cur.execute(create_script)

    insert_script = """
    INSERT INTO aircraft_journey_summary (aircraft_model, registration_number, departure_airport, crew, fuel, load)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cur.execute(insert_script, (
        data['aircraft_model'],
        data['registration_number'],
        data['departure_airport'],
        data['crew'],
        data['fuel'],
        data['load']
    ))

    conn.commit()
    cur.close()
    conn.close()

### **<span style="color:purple;">Database Management with SQLAlchemy</span>**

To manage interactions with the PostgreSQL database, SQLAlchemy will be used. SQLAlchemy provides a high-level API for connecting to the database and performing CRUD operations. Its ORM (Object-Relational Mapping) capabilities will allow developers to work with the database in a Pythonic and intuitive manner

🏅 **<span style="color:green;">Choosing SQLAlchemy:</span>** I chose SQLAlchemy over Django and web2py because it provides a powerful ORM system with advanced query capabilities and flexible database management options, making it ideal for complex and high-performance database tasks



In [ ]:
# https://coderpad.io/blog/development/sqlalchemy-with-postgresql/

# connecting to the postgresql database that was creating in the earlier code, data storage section
engine = create_engine('postgresql')

# Creating new session to interact with the data
Session = sessionmaker(bind=engine)

# Defining the schema - create a class that inherits from a base class
Base = declarative_base()

class AircraftModel(Base):
    __tablename__ = 'aircraft_models'
    id = Column(Integer, primary_key=True)
    model_name = Column(String, unique=True)
    registration_number = Column(String)
    departure_airport = Column(String)
    crew_count = Column(Integer)
    fuel_amount = Column(Integer)
    load_capacity = Column(Integer)

# To actual creat the table aircraft_models above 
Base.metadata.create_all(engine)

# Interacting with the data
session = Session()

### **<span style="color:purple;">Main Function</span>**

In [ ]:
def main(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            try: 
                image_data = preprocess_image(image_path)
                ocr_result = perform_ocr(image_data)
                parsed_data = parse_ocr_result(ocr_result)
                insert_into_db(parsed_data)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

# Execute Main Function
if __name__ == "__main__":
    main("image/folder")